In [1]:
import sys

sys.path.append('../')

In [2]:
from llmsearch.tuner.tuner import EstimatorWrapper

In [3]:
# Autoreload
import sys
from typing import List

import numpy as np
from IPython.display import Audio, display


import nltk
import torch
import numpy as np
import datasets
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, T5ForConditionalGeneration, AutoModelForSeq2SeqLM

device = "cpu"

if torch.backends.mps.is_built() and torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"

nltk.download('punkt')    

from llmsearch.utils.mem_utils import gc_cuda

print(f"Device - {device}")

def beep(duration, frequency=440, rhythm=1):
    sample_rate = 44100  # Standard audio sample rate
    t = np.linspace(0, duration, int(duration * sample_rate), endpoint=False)
    audio_data = np.sin(2*np.pi*frequency*t)  # Generate a sine wave
    audio_data *= np.where(np.arange(len(audio_data)) % rhythm == 0, 1, 0)  # Apply rhythm
    display(Audio(audio_data, rate=sample_rate, autoplay=True))

Device - mps


[nltk_data] Downloading package punkt to /Users/praful932/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
dataset = datasets.load_dataset("samsum")

sample_size = 100
samples_to_tune_on = datasets.Dataset.from_dict(dataset["train"][:sample_size])
samples_to_tune_on = samples_to_tune_on.rename_columns(column_mapping = {'dialogue' : 'X', 'summary' : "y"})

Found cached dataset samsum (/Users/praful932/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
import langchain

X = samples_to_tune_on[0]['X']

pt = langchain.PromptTemplate.from_template("Conversation: {X} Summary:")

pt.format_prompt(X = X).to_string()

"Conversation: Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-) Summary:"

In [10]:
from llmsearch.tuner import Tuner
from llmsearch.utils.mem_utils import get_total_available_ram, get_gpu_information

from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

seed = 42


tuner_ob = Tuner(model = None,tokenizer = tokenizer,dataset = samples_to_tune_on,device = "cpu", batch_size = 16, scorer = None, prompt_template = pt, is_encoder_decoder = False, seed = 42)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [14]:
from sklearn.base import clone

clone(tuner_ob.estimator)

In [15]:
clone??

In [27]:
"""
    Inspired from toma - https://github.com/BlackHC/toma
"""

import gc
import torch

def batch(func,*,batch_size = None,):
    gc_cuda()

    #
#     if batch_size is None and 'batch_size' in kwargs:
#         batch_size = kwargs['batch_size']

    while True:
        try:
            print(f"Im in the decorator - {func.__name__}")
            return func(batch_size)
        except RuntimeError as exception:
            if batch_size > 1 and should_reduce_batch_size(exception):
                batch_size //= 2
                gc_cuda()
            else:
                raise

def gc_cuda():
    """Gargage collect Torch (CUDA) memory."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def should_reduce_batch_size(exception):
    return is_cuda_out_of_memory(exception) or is_cudnn_snafu(exception) or is_out_of_cpu_memory(exception)

def is_cuda_out_of_memory(exception):
    return (
        isinstance(exception, RuntimeError) and len(exception.args) == 1 and "CUDA out of memory." in exception.args[0]
    )

def is_cudnn_snafu(exception):
    # For/because of https://github.com/pytorch/pytorch/issues/4107
    return (
        isinstance(exception, RuntimeError)
        and len(exception.args) == 1
        and "cuDNN error: CUDNN_STATUS_NOT_SUPPORTED." in exception.args[0]
    )

def is_out_of_cpu_memory(exception):
    return (
        isinstance(exception, RuntimeError)
        and len(exception.args) == 1
        and "DefaultCPUAllocator: can't allocate memory" in exception.args[0]
    )

In [1]:
def batch(func):
    """Run inference by deciding appropriate batch size on input
    """
    def inner_wrapper(*args, batch_size, **kwargs):
        while True:
            try:
                print(f"func - {args}, kwargs - {kwargs}, batch_size - {batch_size}")
                res = func(*args,batch_size = batch_size, **kwargs)
                return res
            except RuntimeError as exception:
                raise
                if batch_size > 1 and should_reduce_batch_size(exception):
                    batch_size //= 2
                    gc_cuda()
                else:
                    raise
    
    return inner_wrapper

@batch
def foo(a,b, batch_size = 8):
    """
    a, b - batch of data
    
    default batch size - 8 will try to reduce by half if it doesn't fit in memory
    """
    print(f"in function foo result - {a+b}, batch_size - {batch_size}")
    
foo(2, b = 4, batch_size = 4)

func - (2,), kwargs - {'b': 4}, batch_size - 4
in function foo result - 6, batch_size - 4


In [29]:
@batch
def foo(batch_size, a = 1, b = 2):
    print(f"I'm in function foo - {batch_size}")
    
foo(batch_size = None)

Im in the decorator - foo
I'm in function foo - None


TypeError: 'NoneType' object is not callable